# Diabetes Prediction using PySpark MLlib

In this project, we will build a logistic regression model to classify between diabetic and the non-diabetic patients. After training the model, we assess its performance using relevant metrics to gauge accuracy and effectiveness. The model is saved for future use, ensuring it can be retrieved and deployed in real-world applications to make predictions on new data.

This project has four parts:

- Part 1 - Perform ETL Activity
  - Load a csv dataset
  - Check for null values in each column
  - Replace zero values with mean of the column
  - Store the cleaned data in parquet format
- Part 2 - Build a Logistic Regression Classifier
  - Correlation analysis among the input and the output variables
  - Selection of the input features
  - Split the data into training and test sets
  - Build and train the Logistic Regression Model
- Part 3 - Evaluate the Model
  - Evaluate the model using relevant metrics
- Part 4 - Persist the Model
  - Save the model for future production use
  - Load and verify the stored model

### Preliminaries: Installing libraries and downloading data

Install the required libraries

In [1]:
! pip install pyspark
! pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=d9d38a744d5b6d28d3a5ed62dc1a354946c7fedd4e43fd66009c071b240e3f7c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Clone the required dataset from GitHub

In [2]:
! git clone https://github.com/pregismond/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 13.02 KiB | 13.02 MiB/s, done.


Check if dataset exists

In [3]:
! ls diabetes_dataset

diabetes.csv  new_test.csv


### Importing Libraries

Importing the required libraries

In [4]:
import os
import findspark
import warnings

def warn(*args, **kwargs):
    pass

# Suppress generated warnings
warnings.warn = warn
warnings.filterwarnings("ignore")

findspark.init()

# import functions/Classes for sparkml
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col, filter, mean, when
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

# import functions/Classes for metrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator

### Create a spark session

Ignore any warnings by SparkSession command

In [5]:
spark = SparkSession \
    .builder \
    .appName("Diabetes Prediction") \
    .getOrCreate()

## Tasks

### Part 1 - Perform ETL Activity

Our initial step involves reading the CSV file named `diabetes.csv` into a Spark DataFrame called `diabetes_df`.

Load a csv dataset

* Using the `spark.read.csv` function we load the data into a dataframe
* The `header=True` indicates that there is a header row in our csv file
* The `inferSchema=True` tells spark to automatically determine the data types of the columns

In [6]:
diabetes_df = spark.read.csv("./diabetes_dataset/diabetes.csv", header=True, inferSchema=True)

We then display the structure of the `diabetes_df` DataFrame, including details about all columns and their associated data types.

In [7]:
diabetes_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



Show top 5 rows from the dataset

In [8]:
diabetes_df.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



Show the dimensions of the dataframe (rows, columns)

In [9]:
print((diabetes_df.count(), len(diabetes_df.columns)))

(2000, 9)


Print the value counts for the column `Outcome`

In [10]:
diabetes_df.groupBy("Outcome") \
    .count().withColumnRenamed("count", "Count") \
    .sort("Count", ascending=False) \
    .show()

+-------+-----+
|Outcome|Count|
+-------+-----+
|      0| 1316|
|      1|  684|
+-------+-----+



The `Outcome` column consists of two classes, each indicating whether a patient has diabetes or not:

* **0**: the patient does not have diabetes
* **1**: the patient has diabetes

We can generate descriptive statistics to view some basic statistical details like count, mean, standard deviation, etc.

In [11]:
diabetes_df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

As we can see above, the minimum values for the `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin`, and `BMI` columns are 0, which is an invalid reading. We will replace the zero values in these five columns with their respective mean values. However, before doing so, let’s check for any null or missing values in the dataframe.

Check for null values in each column

In [12]:
for column in diabetes_df.columns:
    null_count = diabetes_df[diabetes_df[column].isNull()].count()
    print(f"{column}: {null_count}")

Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


As you can see, we do not have any missing values for any of the columns present in our dataframe.

Replace zero values with mean of the column

* Replace zero values for the 5 columns from Glucose to BMI with their respective mean values

In [13]:
columns_list = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# Replace zero values with mean of the column
for column in columns_list:
    # Count zero values in the column
    zero_count = diabetes_df.filter(col(column) == 0).count()

    # Calculate mean value of the column and convert to integer
    mean_value = int(diabetes_df.select(mean(col(column))).collect()[0][0])

    # Replace zero values with mean value
    print(f"Zero values in {column}: {zero_count}, Mean value: {mean_value}")
    diabetes_df = diabetes_df.withColumn(column, when(col(column) == 0, mean_value).otherwise(col(column)))

Zero values in Glucose: 13, Mean value: 121
Zero values in BloodPressure: 90, Mean value: 69
Zero values in SkinThickness: 573, Mean value: 20
Zero values in Insulin: 956, Mean value: 80
Zero values in BMI: 28, Mean value: 32


Display the dataframe contents

In [14]:
diabetes_df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


Store the cleaned data in parquet format

* Save the dataframe as `diabetes_cleaned.parquet`

In [15]:
diabetes_df.write.mode("overwrite").parquet("diabetes_cleaned.parquet")

Verify that the parquet file(s) are created

In [16]:
! ls -l diabetes_cleaned.parquet

total 20
-rw-r--r-- 1 root root 18966 Aug  6 00:22 part-00000-6a85822f-23e2-4e03-83e2-06d57b21e1fb-c000.snappy.parquet
-rw-r--r-- 1 root root     0 Aug  6 00:22 _SUCCESS


### Part 2 - Build a Logistic Regression Classifier

First, load data from "diabetes_cleaned.parquet" into a dataframe

In [17]:
diabetes_df = spark.read.parquet("diabetes_cleaned.parquet")

Show total number of rows in the dataset

In [18]:
print(diabetes_df.count())

2000


Determine the correlation among the set of input and output variables

* Correlation is the statistical relationship between two variables, where a change in one variable results in a change in the other.
    * input variables are the columns from `Pregnancies` to `Age`
    * output variable is the `Outcome` column

In [19]:
for column in diabetes_df.columns:
    print(f"Correlation to Outcome for {column} is {diabetes_df.stat.corr('Outcome', column)}")

Correlation to Outcome for Pregnancies is 0.22443699263363961
Correlation to Outcome for Glucose is 0.48796646527321064
Correlation to Outcome for BloodPressure is 0.17171333286446713
Correlation to Outcome for SkinThickness is 0.1659010662889893
Correlation to Outcome for Insulin is 0.1711763270226193
Correlation to Outcome for BMI is 0.2827927569760082
Correlation to Outcome for DiabetesPedigreeFunction is 0.1554590791569403
Correlation to Outcome for Age is 0.23650924717620253
Correlation to Outcome for Outcome is 1.0


As observed above, the Glucose column has the highest correlation value at 0.48, while all other values are below 0.4. This indicates that there are no highly correlated variables. Therefore, we will retain all the input columns as features for the model.

Define `features` selection using VectorAssembler

* Assemble the input columns into a single vector column `features`
* Use all the columns except `Outcome` as input features

In [20]:
assembler = VectorAssembler(
    inputCols=[
        "Pregnancies",
        "Glucose",
        "BloodPressure",
        "SkinThickness",
        "Insulin",
        "BMI",
        "DiabetesPedigreeFunction",
        "Age"
    ],
    outputCol="features"
)

diabetes_transformed_df = assembler.transform(diabetes_df)

Create a new DataFrame `diabetes_final_df` using the existing `diabetes_transformed_df` DataFrame.
* Select only the `features` and `Outcome` columns to isolate the relevant data needed for analysis.

In [21]:
diabetes_final_df = diabetes_transformed_df.select("features","Outcome")

Display the structure of the `diabetes_final_df` DataFrame

In [22]:
diabetes_final_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



Display the dataframe contents

In [23]:
diabetes_final_df.show()

+--------------------+-------+
|            features|Outcome|
+--------------------+-------+
|[2.0,138.0,62.0,3...|      1|
|[0.0,84.0,82.0,31...|      0|
|[0.0,145.0,69.0,2...|      1|
|[0.0,135.0,68.0,4...|      1|
|[1.0,139.0,62.0,4...|      0|
|[0.0,173.0,78.0,3...|      0|
|[4.0,99.0,72.0,17...|      0|
|[8.0,194.0,80.0,2...|      0|
|[2.0,83.0,65.0,28...|      0|
|[2.0,89.0,90.0,30...|      0|
|[4.0,99.0,68.0,38...|      0|
|[4.0,125.0,70.0,1...|      1|
|[3.0,80.0,69.0,20...|      0|
|[6.0,166.0,74.0,2...|      0|
|[5.0,110.0,68.0,2...|      0|
|[2.0,81.0,72.0,15...|      0|
|[7.0,195.0,70.0,3...|      1|
|[6.0,154.0,74.0,3...|      0|
|[2.0,117.0,90.0,1...|      0|
|[3.0,84.0,72.0,32...|      0|
+--------------------+-------+
only showing top 20 rows



Split the data into training and test sets

* We split the data set in the ratio of 70:30. 70% training data, 30% testing data.
* The random_state variable `seed` controls the shuffling applied to the data before applying the split. Pass the same integer for reproducible output across multiple function calls.

In [24]:
(trainingData, testingData) = diabetes_final_df.randomSplit([0.7, 0.3], seed=42)

Create a logistic regression model

* Logistic Regression gives the highest performance for binary classification models.

In [25]:
lr = LogisticRegression(labelCol="Outcome")
model = lr.fit(trainingData)

Display a summary of the trained model, including descriptive statistics of the model's predictions

In [26]:
summary = model.summary
summary.predictions.describe().show()

+-------+------------------+-------------------+
|summary|           Outcome|         prediction|
+-------+------------------+-------------------+
|  count|              1446|               1446|
|   mean|0.3478561549100968|0.26348547717842324|
| stddev|0.4764548683537694| 0.4406758203959858|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



### Part 3 - Evaluate the Model

After training the model, we will assess its accuracy and effectiveness using suitable metrics.

Make predictions on testing data

In [27]:
predictions = model.evaluate(testingData)

Show the predictions

In [28]:
predictions.predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,67.0,76.0,20...|      0|[2.26864766547351...|[0.90624695191016...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.50118195312497...|[0.97072138068613...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.50118195312497...|[0.97072138068613...|       0.0|
|[0.0,78.0,88.0,29...|      0|[2.67406199674440...|[0.93547864095190...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.39829416359169...|[0.91669713242720...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.61115306027706...|[0.93157593147758...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.61115306027706...|[0.93157593147758...|       0.0|
|[0.0,86.0,68.0,32...|      0|[2.57473660615000...|[0.92921786615580...|       0.0|
|[0.0,91.0,68.0,32...|      0|[2.14751320252778...|[0.89543616630293...|    

As you can see, `LogisticRegression` has added three additional columns as predictions:

- **rawPrediction**: This is the raw prediction for each possible label and represents the raw output of the logistic regression classifier.
- **probability**: This is the result of applying logistic regression to this raw prediction.
- **prediction**: This is the corresponding class label that the model has predicted.


Use the `BinaryClassificationEvaluator` to evaluate the overall performance of the model

In [29]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Outcome")
accuracy = evaluator.evaluate(model.transform(testingData))
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8464


### Part 4 - Persist the Model

Save the model for future use, ensuring that it can be stored and retrieved later. This allows us to deploy the trained model in real-world applications and make predictions on new data.

* Save the model as "diabetes_model"

In [30]:
# Create folder to save model
! mkdir -p diabetes_model

# Persist the model to the path "./diabetes_model/"
model.write().overwrite().save("./diabetes_model/")

Load the model from the folder "diabetes_model"

In [31]:
loaded_model = LogisticRegressionModel.load("./diabetes_model/")

Read the csv file named `new_test.csv` into a Spark DataFrame called `new_test_df`

In [32]:
new_test_df = spark.read.csv("./diabetes_dataset/new_test.csv", header=True, inferSchema=True)

Display the structure of the `new_test_df` DataFrame, including details about all columns and their associated data types.

In [33]:
new_test_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)



Here we can see that we have similar input features as before. However, one thing to notice is that we don't have the output column `Outcome` because this dataset is unlabelled. We'll use the loaded model to predict diabetes on this data.

Assemble the input columns into a single vector column `features`

In [34]:
new_test_transformed_df = assembler.transform(new_test_df)

Display the structure of the `new_test_transformed_df` DataFrame

In [35]:
new_test_transformed_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)



Here we have an additional `features` column as a vector.

Use `loaded_model` to make predictions on test data

In [36]:
predictions = loaded_model.transform(new_test_transformed_df)
predictions.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



Here we got an additional 3 columns: `rawPrediction`, `probability`, and `prediction`. The `prediction` column contains the main class level as either 0 or 1.

Show the predictions

* Display only the `features` column and `prediction`

In [37]:
predictions.select("features","prediction").show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[1.0,190.0,78.0,3...|       1.0|
|[0.0,80.0,84.0,36...|       0.0|
|[2.0,138.0,82.0,4...|       1.0|
|[1.0,110.0,63.0,4...|       1.0|
+--------------------+----------+



We have a total of four input features, and our model has made certain predictions on the input data. A prediction of 1 indicates that a patient is diabetic, while a prediction of 0 indicates that a patient is a non-diabetic.


### Stop Spark Session

In [38]:
spark.stop()

## Change Log


|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2024-08-03  | 0.1  | Pravin Regismond | Initial Version |

Copyright © 2024 Pravin Regismond. All rights reserved.